# [실습] LangChain Tool Call과 Agent

# 📊 LangChain + Gemini + 검색 도구 연동, 쉽게 큰 틀부터 이해하기

우리는 지금 AI에게 **"질문을 이해하고, 필요하면 검색해서, 결과를 요약해주는 역할"** 을 맡기려 합니다.

이 구조를 이해하기 위해선, 복잡한 코드를 보기 전에 **큰 흐름부터** 이해하는 것이 가장 중요합니다.  
여기선 전체 흐름을 6단계로 나누어 설명합니다.

---

## 🪜 **단계 1: AI 모델 준비 (LLM 불러오기)**

- 우리는 **Gemini라는 AI 모델**을 사용할 겁니다.
- 이 모델은 말귀를 알아듣고 문장을 생성해주는 역할을 합니다.
- 이때, 너무 자주 질문하면 에러가 나기 때문에 **속도 제한 장치(Rate Limiter)**도 함께 설정합니다.

📌 **비유하자면?**  
AI는 비서, 하지만 바빠서 1분에 10번만 응답 가능하니, 스케줄 관리자가 필요해요.

---

## 🔌 **단계 2: 외부 기능 연결 (검색 도구 연동)**

- AI는 모든 걸 알진 못합니다. 그래서 **인터넷 검색 기능**을 연결해야 합니다.
- 여기선 Tavily라는 검색 엔진을 연결합니다.
- 이 기능은 마치 **'AI의 눈과 귀'**처럼, 인터넷에서 정보를 가져오는 역할을 합니다.

📌 **비유하자면?**  
비서가 모르는 건 인터넷으로 검색해서 알려주는 구조!

---

## 🧠 **단계 3: 프롬프트로 질문 양식 만들기**

- AI가 질문을 잘 이해하도록 **질문 양식(프롬프트)**을 구성합니다.
- 예: “질문을 5문장 이내로, 항목별로 정리해줘” 라는 가이드라인을 미리 줍니다.
- 이 구조를 통해 **AI의 답변 스타일과 길이**를 조정할 수 있어요.

---

## 🔗 **단계 4: AI에게 '이 도구 써도 돼' 하고 알려주기**

- 도구를 만들어놨다고 AI가 바로 쓰는 건 아닙니다.
- 그래서 명시적으로 **“이 도구를 써도 됩니다”** 하고 AI에게 알려줍니다.
- 이 과정을 **도구 바인딩(bind_tools)** 이라고 해요.

📌 **비유하자면?**  
비서에게 “검색기 써도 돼” 하고 툴박스를 열어주는 과정입니다.

---

## 🔍 **단계 5: AI가 판단해서 검색하고, 정리해서 답변**

- 사용자가 질문을 하면, AI는 먼저 **스스로 판단**합니다:
  - 그냥 답할 수 있는가?
  - 검색이 필요한가?

- 필요하다면 도구를 호출해서 검색하고,
- 검색 결과를 받아 다시 AI가 읽고 **최종 답변을 정리**합니다.

📌 **비유하자면?**  
비서가 “이건 검색이 필요하겠군요” → 검색 → 결과 정리 → 보고

---

## 🧩 **단계 6: 전체 흐름을 함수로 묶어 자동화하기**

- 지금까지 단계를 모두 하나의 함수로 정리합니다.
- 즉, 질문을 넣으면:
  1. AI가 판단하고
  2. 검색하고
  3. 정리해서 답변까지 주는 과정을 자동으로 실행합니다.

📌 **비유하자면?**  
비서를 호출하면 필요한 정보를 알아서 검색하고 요약해서 보고해주는 **자동화된 AI 조수** 완성!

---

### ✅ 요약

| 단계 | 설명 |
|------|------|
| 1단계 | Gemini AI 모델 로딩 |
| 2단계 | 검색 도구 Tavily 연동 |
| 3단계 | 프롬프트(질문 양식) 만들기 |
| 4단계 | AI에게 도구 사용 허용 |
| 5단계 | AI가 판단 → 검색 → 답변 정리 |
| 6단계 | 전체 과정을 자동으로 처리하는 함수 만들기 |

---

💡 이렇게 전체 흐름을 이해하고 나면, 코드 하나하나가 어떤 역할인지 **훨씬 명확하게 보이게 됩니다.**


In [1]:
# ✅ 필수 라이브러리 설치
# LangChain과 Google Gemini, Tavily 검색을 위한 라이브러리를 설치합니다.
!pip install -U langchain_google_genai langchain_core langchain_community tavily-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 0.3.59
    Uninstalling langchain-core-0.3.59:
      Successfully uninstalled langchain-core-0.3.59
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are insta

# ✅ Google Gemini API 키 설정 (https://aistudio.google.com/apikey)

In [2]:

# ✅ Google Gemini API 키 설정 (API 키는 Google AI Studio에서 발급 가능)
# Gemini API를 사용하려면 발급받은 키를 환경변수로 등록해야 합니다.
import os
# os.environ['GOOGLE_API_KEY'] = ''

# ✅ 요청 제한 설정: 무료 버전은 분당 10회 제한이 있으므로 제한 설정이 필수입니다
# 포인트: 무료 요금제는 요청이 너무 많으면 오류가 나므로, RateLimiter로 속도를 조절합니다.
from langchain_core.rate_limiters import InMemoryRateLimiter
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.167,  # 초당 약 0.167번 → 분당 10회 제한에 맞춤
    check_every_n_seconds=0.1,  # 0.1초마다 요청 가능 여부를 체크
    max_bucket_size=10          # 최대 대기 요청 수 10개
)



In [3]:
# ✅ Gemini 모델 로드: 응답 속도가 빠른 Flash 모델 사용
# 포인트: Gemini는 Google의 최신 생성형 AI입니다. 여기선 빠른 응답을 위해 Flash 버전을 사용합니다.
from langchain_google_genai import ChatGoogleGenerativeAI
language_model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",  # 응답이 빠른 경량 버전
    rate_limiter=rate_limiter      # 위에서 만든 요청 제한 설정 적용
)

# ✅ LLM 동작 테스트
# 포인트: AI에게 간단한 질문을 던져, 모델이 잘 연결되어 있는지 확인합니다.
language_model.invoke("안녕하세요, 간단하게 자기소개 해주세요.")



AIMessage(content='안녕하세요! 저는 사용자의 질문에 답변하고, 요청을 수행하도록 설계된 Google의 대규모 언어 모델입니다. 다양한 종류의 텍스트를 이해하고 생성할 수 있으며, 아직 개발 중이지만 끊임없이 배우고 발전하고 있습니다. 무엇을 도와드릴까요? 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-exp', 'safety_ratings': []}, id='run--d709c9cb-6d14-475c-a54e-74e6dd4eef6c-0', usage_metadata={'input_tokens': 15, 'output_tokens': 83, 'total_tokens': 98, 'input_token_details': {'cache_read': 0}})

In [4]:
# ✅ 메시지를 통한 대화 흐름 구성
from langchain_core.messages import HumanMessage, SystemMessage

intro_prompt = SystemMessage(content="""
아래 사용자 질문에 대해 간결하고 명확하게 답변하세요.
응답은 5문장을 넘지 않도록 하고, 항목별로 정리하세요.
""")
user_query = HumanMessage(content="당신은 어떤 역할을 하나요?")
messages = [intro_prompt, user_query]

response = language_model.invoke(messages)
print(response.content)

# ✅ 프롬프트 템플릿 활용
from langchain.prompts import ChatPromptTemplate
structured_prompt = ChatPromptTemplate.from_messages([
    ("system", "사용자의 질문에 대해 짧고 명확한 설명을 제공하세요. 5문장을 넘지 않게 하고, 핵심 위주로 정리해주세요."),
    ("user", "[질문]: {inquiry}")
])

qa_chain = structured_prompt | language_model
print(qa_chain.invoke({"inquiry": "파이썬의 주요 특징이 뭐야?"}).content)

저는 대규모 언어 모델입니다.

*   구글에서 개발되었습니다.
*   다양한 종류의 텍스트를 이해하고 생성할 수 있습니다.
*   질문에 답하거나, 텍스트를 요약하거나, 번역하는 등 다양한 작업을 수행합니다.
*   아직 개발 중인 단계이며, 지속적으로 학습하고 있습니다.
*   제공하는 정보가 항상 정확하거나 완전하지 않을 수 있습니다.
파이썬은 배우기 쉬운 문법과 다양한 활용도로 유명한 프로그래밍 언어입니다. 코드가 간결하고 읽기 쉬워서 초보자에게 적합하며, 다양한 운영체제에서 실행됩니다. 방대한 라이브러리를 제공하여 여러 분야에서 활용 가능하며, 웹 개발, 데이터 분석, 머신러닝 등에 널리 쓰입니다. 또한, 객체 지향 프로그래밍을 지원하여 코드 재사용성과 유지보수성을 높여줍니다. 파이썬은 생산성이 높고 빠르게 개발할 수 있다는 장점이 있습니다.


Tavily Search (http://app.tavily.com/)

Tavily는 AI 기반의 검색 엔진입니다. 계정별 월 1000개의 무료 사용량을 지원합니다.      
Tavily Search는 URL과 함께 내용의 간단한 요약을 지원하는 것이 특징입니다.

In [5]:
# ✅ Tavily 검색 도구 연동
# os.environ['TAVILY_API_KEY'] = ''
from langchain_community.tools.tavily_search import TavilySearchResults
web_search_tool = TavilySearchResults(
    max_results=5,
    include_answer=True,
    include_raw_content=True
)

# ✅ 검색 도구 테스트
search_result = web_search_tool.invoke("파이썬과 자바의 차이점")
print(search_result)


[{'title': '자바와 파이썬의 비교: 장단점과 활용 사례', 'url': 'https://f-lab.kr/insight/java-python-comparison-20240914', 'content': '자바와 파이썬의 비교\n자바와 파이썬은 각각의 장단점을 가진 두 가지 인기 있는 프로그래밍 언어입니다. 자바는 강타입 언어로, 코드의 안정성과 유지보수성을 높이는 데 유리합니다. 반면 파이썬은 동적 타이핑을 지원하여 코드 작성이 간편하고 빠르게 개발할 수 있는 장점이 있습니다.\n왜냐하면 자바는 컴파일 시점에 타입을 체크하여 오류를 사전에 방지할 수 있기 때문입니다. 반면 파이썬은 런타임 시점에 타입을 체크하여 유연한 코딩이 가능합니다.\n자바는 대규모 시스템 개발에 적합하며, 안정성과 성능이 중요한 프로젝트에서 많이 사용됩니다. 반면 파이썬은 데이터 분석, 머신러닝, 웹 개발 등 다양한 분야에서 활용됩니다.\n자바는 JVM(Java Virtual Machine) 위에서 실행되며, 플랫폼 독립성을 제공합니다. 반면 파이썬은 인터프리터 언어로, 다양한 운영체제에서 쉽게 실행할 수 있습니다. [...] 자바는 강력한 라이브러리와 프레임워크를 제공하여 개발 생산성을 높일 수 있으며, 플랫폼 독립성을 제공합니다. 이를 통해 다양한 운영체제에서 동일한 코드를 실행할 수 있습니다.\n자바와 파이썬 중 어떤 언어를 선택할지는 프로젝트의 요구사항과 개발자의 선호도에 따라 달라질 수 있습니다. 따라서 프로젝트의 요구사항을 고려하여 적절한 언어를 선택하는 것이 중요합니다.\n결론\n자바와 파이썬은 각각의 장단점을 가진 두 가지 인기 있는 프로그래밍 언어입니다. 자바는 강타입 언어로, 코드의 안정성과 유지보수성을 높이는 데 유리합니다.\n파이썬은 동적 타이핑을 지원하여 코드 작성이 간편하고 빠르게 개발할 수 있는 장점이 있습니다. 따라서 프로젝트의 요구사항에 따라 적절한 언어를 선택하는 것이 중요합니다.\n자바는 대규모 시스템 개발에 적합하며, 안정성과 성능이 중요한 프로젝

In [6]:
# ✅ 도구 바인딩
language_model_with_tools = language_model.bind_tools([web_search_tool])
qa_chain_with_tool = structured_prompt | language_model_with_tools

# ✅ 도구 호출 포함 질문
tool_response = qa_chain_with_tool.invoke({"inquiry": "여름에 입을 트렌디한 빈티지 옷을 검색해줘!"})
print(tool_response.content)

[답변]: 여름에 입을 트렌디한 빈티지 옷을 검색해 드릴게요.


In [7]:
# ✅ Tool Call 분석 및 실행
from langchain_core.messages import AIMessage

tool_mapping = {'tavily_search_results_json': web_search_tool}

executed_tool_result = None

if hasattr(tool_response, "tool_calls") and tool_response.tool_calls:
    tool_call = tool_response.tool_calls[0]
    tool_name = tool_call['name'] if isinstance(tool_call, dict) else tool_call.name
    tool_args = tool_call['args'] if isinstance(tool_call, dict) else tool_call.args
    tool_executor = tool_mapping.get(tool_name)

    if tool_executor:
        executed_tool_result = tool_executor.invoke(tool_args)
        print(executed_tool_result)
    else:
        print("❌ 도구 이름이 일치하지 않습니다.")
else:
    print("❌ 도구 호출 정보가 없습니다.")

[{'title': 'Summer Vintage Outfits 2024: 34+ Ideas You Will LOVE', 'url': 'https://vintaked.com/summer-vintage-outfits/', 'content': 'How can I style vintage pieces for a modern summer look?\nMix vintage items with contemporary pieces, experiment with layering, accessorize with modern jewelry or shoes to create a unique outfit.\nWhat are some popular vintage summer outfit trends?\nMidi dresses, high-waisted shorts, floral prints, oversized sunglasses, straw hats, and platform sandals are all popular vintage summer trends to try. [...] by Pinterest\nOh my gosh, I absolutely adore this summer vintage outfit! The light denim overalls are such a timeless piece, and they give off that effortlessly cool vibe. Paired with a simple white tank top, it’s the perfect blend of comfort and style for those warm, sunny days. Plus, the rolled-up shorts add a cute, playful touch that makes it a must-have for any vintage lover’s wardrobe!\nCharming Summer Vintage Outfit [...] by Pinterest\nOh, I absolut

In [8]:
from langchain_core.messages import HumanMessage, AIMessage

if executed_tool_result:
    # 리스트의 첫 번째 항목을 사용
    first_result = executed_tool_result[0] if isinstance(executed_tool_result, list) and executed_tool_result else {}
    result_text = first_result.get("answer") or first_result.get("content", "검색 결과가 없습니다.")

    conversation = [
        HumanMessage(content="여름에 입을 트렌디한 빈티지 옷을 검색해줘!"),
        AIMessage(content=tool_response.content),
        AIMessage(content=result_text)
    ]

    final_result = language_model_with_tools.invoke(conversation)
    print(final_result.content)


 [...] by Pinterest
This summer vintage outfit is absolutely charming! The loose-fitting linen pants offer ultimate comfort and breathability, perfect for those hot days. Paired with the simple white top, it creates a clean, effortless look. The addition of the wide-brimmed hat not only protects from the sun but also adds a touch of vintage elegance. It’s the ideal ensemble for a stroll through the park or a casual day out.
Summer Vintage Outfit Inspiration [...] by Pinterest
This summer vintage outfit is absolutely enchanting! The flowy, floral maxi dress exudes a timeless elegance, perfect for warm, sunny days. Paired with delicate sandals and minimal jewelry, it creates an effortlessly chic look. The dress’s soft fabric and vibrant pattern capture the essence of summer, making it a must-have for any vintage lover’s wardrobe.
Summer Vintage Outfit Inspiration [...] by Pinterest
This summer vintage outfit is absolutely charming! The high-waisted denim shorts paired with the tucked-in 

In [9]:
# ✅ 전체 워크플로우 함수화
# 위 과정을 자동화하는 함수. 질문을 넣으면 알아서 판단, 검색, 응답까지 수행합니다.
from langchain_core.messages import ToolMessage

def query_with_tool_support(llm, question, tools=[web_search_tool]):
    tool_dict = {tool.name: tool for tool in tools}  # 도구 이름으로 매핑
    model_with_tools = llm.bind_tools(tools)         # 모델에 도구 바인딩

    print("[질문 입력]:", question)
    query_message = HumanMessage(content=question)
    response = model_with_tools.invoke([query_message])

    if response.tool_calls:
        # 도구 호출이 포함된 경우 → 실제로 실행 후 응답 생성
        selected_tool = tool_dict[response.tool_calls[0]['name']]
        tool_result = selected_tool.invoke(response.tool_calls[0])

        full_conversation = [query_message, response, tool_result]
        final_answer = model_with_tools.invoke(full_conversation)
    else:
        # 도구 없이도 답할 수 있을 경우, 바로 응답 반환
        final_answer = response

    return final_answer.content

# ✅ 실행 예시
# 질문을 함수에 넣기만 하면 AI가 알아서 판단하고 필요한 작업을 수행합니다.
print(query_with_tool_support(language_model, "애플의 2024년 주요 제품은 무엇인가요?"))

[질문 입력]: 애플의 2024년 주요 제품은 무엇인가요?
 2024년에 예상되는 애플의 주요 제품은 다음과 같습니다:

*   아이폰 16/16 플러스 & 16 프로/울트라
*   애플 워치 X
*   OLED 아이패드 프로
*   M3 맥북 에어
*   에어팟 4세대
*   에어팟 맥스 2세대
